# Ticketing system

In [57]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from pandas import json_normalize
import nltk
from nltk.corpus import stopwords
import string
%matplotlib inline
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns",None)

In [58]:
#import the data
df = open('client_data.json')

In [61]:
data = json.load(df)
df = pd.json_normalize(data)

In [62]:
df.head(4)

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,_source.consumer_disputed,_source.product,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,N/A,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,N/A,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,N/A,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,complaint-public-v2,complaint,2673060,0.0,None,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,N/A,Mortgage,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided


In [63]:
df.columns

Index(['_index', '_type', '_id', '_score', '_source.tags', '_source.zip_code',
       '_source.complaint_id', '_source.issue', '_source.date_received',
       '_source.state', '_source.consumer_disputed', '_source.product',
       '_source.company_response', '_source.company', '_source.submitted_via',
       '_source.date_sent_to_company', '_source.company_public_response',
       '_source.sub_product', '_source.timely',
       '_source.complaint_what_happened', '_source.sub_issue',
       '_source.consumer_consent_provided'],
      dtype='object')

In [64]:
print(len(df))

78313


In [65]:
df.shape

(78313, 22)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78313 entries, 0 to 78312
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   _index                             78313 non-null  object 
 1   _type                              78313 non-null  object 
 2   _id                                78313 non-null  object 
 3   _score                             78313 non-null  float64
 4   _source.tags                       10900 non-null  object 
 5   _source.zip_code                   71556 non-null  object 
 6   _source.complaint_id               78313 non-null  object 
 7   _source.issue                      78313 non-null  object 
 8   _source.date_received              78313 non-null  object 
 9   _source.state                      76322 non-null  object 
 10  _source.consumer_disputed          78313 non-null  object 
 11  _source.product                    78313 non-null  obj

In [67]:
df.dtypes

_index                                object
_type                                 object
_id                                   object
_score                               float64
_source.tags                          object
_source.zip_code                      object
_source.complaint_id                  object
_source.issue                         object
_source.date_received                 object
_source.state                         object
_source.consumer_disputed             object
_source.product                       object
_source.company_response              object
_source.company                       object
_source.submitted_via                 object
_source.date_sent_to_company          object
_source.company_public_response       object
_source.sub_product                   object
_source.timely                        object
_source.complaint_what_happened       object
_source.sub_issue                     object
_source.consumer_consent_provided     object
dtype: obj

In [68]:
# creating new dataset using required columns
data = df[['_source.sub_product','_source.complaint_what_happened','_source.sub_issue']]
data.columns = ['sub_product','complaint_what_happened','sub_issue']
data.head(10)

,sub_product,complaint_what_happened,sub_issue
0,Credit card debt,,Debt is not yours
1,Credit card debt,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt
2,General-purpose credit card or charge card,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card
3,Conventional home mortgage,,None
4,General-purpose credit card or charge card,,Charged too much interest
5,Checking account,,Problem using a debit or ATM card
6,Checking account,,Problem using a debit or ATM card
7,Conventional home mortgage,,None
8,Checking account,,Company closed your account
9,Checking account,,Cashing a check


In [69]:
#checking how many rows are there
len(data[data['complaint_what_happened'] == ""])

57241

In [70]:
data['complaint_what_happened'].shape


(78313,)

In [71]:
data['complaint_what_happened'].replace('',np.nan,inplace=True)

In [72]:
data.dropna(subset=['complaint_what_happened'],inplace=True)

In [73]:
data['complaint_what_happened'].shape

(21072,)

# Text processing

In [74]:
# removing stopwords 

stop_words = set(stopwords.words('English'))

def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

data['sub_product'] = data['sub_product'].apply(lambda text: cleaning_stopwords(text))
data['complaint_what_happened'] = data['complaint_what_happened'].apply(lambda text: cleaning_stopwords(text))
data['sub_issue'] = data['sub_issue'].apply(lambda text: cleaning_stopwords(text))
data.head()

,sub_product,complaint_what_happened,sub_issue
1,Credit card debt,Good morning name XXXX XXXX I appreciate could...,Didn't receive enough information verify debt
2,General-purpose credit card charge card,I upgraded XXXX XXXX card XX/XX/2018 told agen...,Problem rewards credit card
10,Other personal consumer report,"Chase Card reported XX/XX/2019. However, fraud...",Information belongs someone else
11,Credit reporting,"On XX/XX/2018, trying book XXXX XXXX ticket, I...",Information belongs someone else
14,Checking account,grand son give check {$1600.00} deposit chase ...,Funds handled disbursed instructed


# removing punctuation

In [75]:
regular_punct = list(string.punctuation)


def remove_punctuation(text,punct_list):
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip()
data['complaint_what_happened'] = data['complaint_what_happened'].apply(lambda text: remove_punctuation(text,regular_punct))


In [76]:
data

,sub_product,complaint_what_happened,sub_issue
1,Credit card debt,Good morning name XXXX XXXX I appreciate could...,Didn't receive enough information verify debt
2,General-purpose credit card charge card,I upgraded XXXX XXXX card XX XX 2018 told agen...,Problem rewards credit card
10,Other personal consumer report,Chase Card reported XX XX 2019 However fraud...,Information belongs someone else
11,Credit reporting,On XX XX 2018 trying book XXXX XXXX ticket I...,Information belongs someone else
14,Checking account,grand son give check 1600 00 deposit chase ...,Funds handled disbursed instructed
...,...,...,...
78303,General-purpose credit card charge card,After Chase Card customer well decade offered...,Didn't receive advertised promotional terms
78309,General-purpose credit card charge card,On Wednesday XX XX XXXX I called Chas XXXX X...,Other problem
78310,Checking account,I familiar XXXX pay understand great risk prov...,Transaction authorized
78311,General-purpose credit card charge card,I flawless credit 30 yrs I ve Chase credit ca...,Problem balance transfer


In [77]:
#import regular expression
import re

In [78]:
#removing numeric data

def remove_numeric(text):
    
    return re.sub(r'[0-9]*','',text)


In [79]:
data['complaint_what_happened']=data['complaint_what_happened'].apply(lambda text: remove_numeric(text))

In [80]:
data

,sub_product,complaint_what_happened,sub_issue
1,Credit card debt,Good morning name XXXX XXXX I appreciate could...,Didn't receive enough information verify debt
2,General-purpose credit card charge card,I upgraded XXXX XXXX card XX XX told agent up...,Problem rewards credit card
10,Other personal consumer report,Chase Card reported XX XX However fraudulen...,Information belongs someone else
11,Credit reporting,On XX XX trying book XXXX XXXX ticket I cam...,Information belongs someone else
14,Checking account,grand son give check deposit chase accoun...,Funds handled disbursed instructed
...,...,...,...
78303,General-purpose credit card charge card,After Chase Card customer well decade offered...,Didn't receive advertised promotional terms
78309,General-purpose credit card charge card,On Wednesday XX XX XXXX I called Chas XXXX X...,Other problem
78310,Checking account,I familiar XXXX pay understand great risk prov...,Transaction authorized
78311,General-purpose credit card charge card,I flawless credit yrs I ve Chase credit card...,Problem balance transfer


# Feature Extraction

In [81]:
vectorizer = TfidfVectorizer()

In [82]:
f=vectorizer.fit_transform(data.complaint_what_happened)

In [83]:
data

,sub_product,complaint_what_happened,sub_issue
1,Credit card debt,Good morning name XXXX XXXX I appreciate could...,Didn't receive enough information verify debt
2,General-purpose credit card charge card,I upgraded XXXX XXXX card XX XX told agent up...,Problem rewards credit card
10,Other personal consumer report,Chase Card reported XX XX However fraudulen...,Information belongs someone else
11,Credit reporting,On XX XX trying book XXXX XXXX ticket I cam...,Information belongs someone else
14,Checking account,grand son give check deposit chase accoun...,Funds handled disbursed instructed
...,...,...,...
78303,General-purpose credit card charge card,After Chase Card customer well decade offered...,Didn't receive advertised promotional terms
78309,General-purpose credit card charge card,On Wednesday XX XX XXXX I called Chas XXXX X...,Other problem
78310,Checking account,I familiar XXXX pay understand great risk prov...,Transaction authorized
78311,General-purpose credit card charge card,I flawless credit yrs I ve Chase credit card...,Problem balance transfer


In [84]:
vocabluary=np.array(vectorizer.get_feature_names())

In [85]:
vocabluary.shape

(27981,)

In [86]:
from sklearn.decomposition import NMF
n_topics = 5
random_state = 42
nmf = NMF(n_components=n_topics, random_state=random_state)
nmf.fit(f)

NMF(n_components=5, random_state=42)

# Topic Modelling with the help of NMF

In [87]:
## Get words from each topic 
n_top_words=10
def get_nmf_topics(n_top_words,vocabulary,nmf):
    for i in range(num_topics):
        words_ids = model.components_[i].argsort()[-1:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
    return pd.DataFrame(word_dict);

In [ ]:
get_nmf_topics(10,vocabulary,nmf)